In [ ]:
#libraries

import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import talib as ta
import seaborn as sns
#from datetime import datetime
import glob
import os
#import subprocess
sns.set_theme()

pd.set_option('display.max_columns', None);pd.set_option('display.max_rows', None)


In [ ]:
def plot_with_mpl(root,tick,day=60,sty='classic',typ='candle'):
    "returns the candle stick of a stock tick for the given root file."

    df=pd.read_csv(root+'/'+tick+'.csv').tail(day)
    df['Date']=df['Date'].apply(lambda x:x.split()[0])

    df['rsi']=ta.RSI(df['Close']);df['sma-5']=ta.SMA(df['Close'],5);df['sma-10']=ta.SMA(df['Close'],10)
    #df.fillna(0,inplace=True)
    apds = [ 
         mpf.make_addplot((df['rsi']),panel=1,color='r',secondary_y=True,type='line',width=0.5),
         #mpf.make_addplot((df['sma-5']),color='g',type='line',width=0.2),
         #mpf.make_addplot((df['sma-10']),color='r',type='line',width=0.2)
       ]
    df.Volume=df.Volume/10**6              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,axes=mpf.plot(df,type=typ,mav=(5,21),volume=True,tight_layout=False,figratio=(80,36),figscale=1,
                    style=sty,returnfig=True,addplot=apds,scale_width_adjustment=dict(volume=0.6,candle=0.8,lines=0.4))#,update_width_config=dict(candle_linewidth=1.75,candle_width=0.1))
    axes[0].legend([" "," ","5-SMA","21-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    axes[0].set_title(tick)
    mpf.show()

# def make_clickable(url, name):
#     return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)

def get_yahoo_link(ticker):
    """
    Get the yahoo link
    """
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)
    url="https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch"
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,ticker)
    return url.format(ticker,ticker)

def short_industry(x):
    temp=x.split()
    if len(temp)>2:
        return temp[0]+"-"+temp[1]
    else:
        return x


In [ ]:
def plot_all(j,plot=True):
    #j=0 #nano
    df=pd.read_csv(files[j]).drop(columns=['SMA150','SMA200PAST20','NAME']).sort_values(by='CP').reset_index(drop=True)
    df['YAHOO']=df['TICK'].apply(lambda x:get_yahoo_link(x))#.apply(lambda x: yahoo_clickable(x, 'yahoo'))

    #df_sector=reset_index(df_sector)
    print(df.to_string())
    tick=df['TICK'].to_list()
    #sec=df_sector['SECTOR'].to_list()
    root=roots[j]
    if plot:
        for c,i in enumerate(tick):
        #file=root.format(cat[c],i)
        #print(f"file: {file}")
            print(f"\n\n{df[df['TICK']==i].to_string()}\n")
            plot_with_mpl(root,i,sty='default')

In [ ]:
#create yfinance portfolio sample
def yahoo_portfolio(df,save_name):
    """
    returns the yfinance compatable portfolio template for given df name of save file
    """
    cols=['Symbol','Current Price','Date', 'Time', 'Change', 'Open', 'High', 'Low', 'Volume', 'Trade Date',
          'Purchase Price','Quantity','Commission','High Limit','Low Limit','Comment']

    sym=list(df.TICKER)
    dic_val=[sym]+14*[None] #14 None
    my_dict=dict(zip(cols,dic_val)) #dictionary
    my_df=pd.DataFrame.from_dict(my_dict)
    my_df.to_csv(save_name+".csv")
    print(f"Created the yahoo template: {save_name}.csv from data frame !")

In [ ]:
root='/home/thakur/test/combined/{}'
file_path='/home/thakur/test/combined/mark*.csv'
files = sorted(glob.iglob(file_path), key=os.path.getctime, reverse=True)[:6]
roots=['nano','micro','small','medium','large','mega']
roots=['/home/thakur/test/'+i for i in roots]
files

In [ ]:
all_nasdaq='all_nasdaq.csv'
all_mark='all_mark.csv'

# ALL NASDAQ

In [ ]:
df_nasdaq=pd.read_csv(root.format(all_nasdaq))
df_nasdaq.head()

# MARK

In [ ]:
df_mark=pd.read_csv(root.format(all_mark))
df_mark.head()

# SHAPE

In [ ]:
print(df_mark.shape)

In [ ]:
test=df_mark.TICK.to_list()
df=df_nasdaq[df_nasdaq.TICKER.isin(test)]\
.reset_index(drop=True)
df.head(10)

# HH & HL & HC 

In [ ]:
# select the data from Mark
#category=['mega','medium','large','small','micro','nano']
con='HH & HL & HC & HV'
selected=['mega','medium','large','small','micro','nano']
df=df[df.CATEGORY.isin(selected)]\
.query(con)\
.sort_values('CP')\
.reset_index(drop=True)
#df_mark[list(df_mark.CATEGORY.unique()) in ['mega']]
#df_mark[df_mark.CATEGORY.unique in ['medium']]# | df_mark.CATEGORY=='large' | df_mark.CATEGORY=='small']
df.head(10)

In [ ]:
#choose=['TICKER', 'CP','HH','HL','HV', 'RSI', 'SMA21', 'SMA50', 'SMA200','VOL', 'AVGVOL', '%CHG', 'ATR', 'SECTOR',  'IPO','CATEGORY', 'ylink']
#print(choose)
df.head(20)

# By category

In [ ]:
present=df.CATEGORY.unique()
print(f" {present}")

In [ ]:
drop_cols=list(df.columns)[3:8]+['HV']+['EMA8']+['INDUSTRY']+['COUNTRY']
print(f"Dropped Columns: {drop_cols}")
df_t=df.drop(columns=drop_cols)
df_t.head()

In [ ]:
grouped=df.groupby(by='CATEGORY')
for i,cat in enumerate(present,start=1):
    print(f"\nWorking with: {i}/{len(present)} => {cat}\n")
    df_temp=grouped.get_group(cat).reset_index(drop=True)
    df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    df_temp.index+=1
    print(f"{df_temp.to_string()}")

In [ ]:
# df=df_nasdaq.query('HH & HL & HC')\
# .sort_values(by='CP')\
# .reset_index(drop=True)

df['ylink']=df['TICKER'].apply(lambda x:get_yahoo_link(x))
#df['INDUSTRY']=df['INDUSTRY'].apply(lambda x:short_industry(x))
df.head(10)

In [ ]:
get=['TICKER','CP','RSI','VOL','AVGVOL','%CHG','ATR','CATEGORY','ylink']
print(df[get].to_string())

In [ ]:
#df.COUNTRY.value_counts()


In [ ]:
print(f"Total length : {df.shape[0]}")

In [ ]:
int_list=['SECTOR','CATEGORY','IPO']
for c,i in enumerate(int_list):
    if c==0:print(20*"==")
    print(f"{i:^40}\n\n{df[i].value_counts()}\n")
    print(20*"==")

In [ ]:
df.columns

In [ ]:
slc=['TICKER', 'CP', 'RSI',
       'SMA5', 'SMA21', 'SMA50', 'SMA200', 'VOL', 'AVGVOL', '%CHG', 'ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY', 'ylink']

In [ ]:
yahoo_portfolio(df,"yahoo_mark")

In [ ]:
tickers=df.TICKER.to_list()
r='/home/thakur/test/{}'
#ignore=['micro','nano','mega']
for i in tickers:
    root=df.loc[df.TICKER==i,"CATEGORY"].values[0]
    #if root=='micro' or root=='nano':continue #ignoring micro and nano
    #if root in ignore:continue
    pst=df.loc[df.TICKER==i][slc].to_string()
    print(f"{pst}")
    #print(f"{df.loc[df.TICKER==i,"CATEGORY"]}")
    root=r.format(root)
    #print(f"root: {root}")
    plot_with_mpl(root,i,sty='default')

# MARK MEGA

In [ ]:
plot=True
plot_all(5,plot)

# MARK LARGE

In [ ]:
plot=True
plot_all(4,plot)

# MARK MEDIUM

In [ ]:
plot=True
plot_all(3,plot)

# MARK SMALL

In [ ]:
plot=True
plot_all(2,plot)

# MARK MICRO



In [ ]:
plot=True
plot_all(1,plot)

# MARK NANO

In [ ]:
plot=True
plot_all(0,plot)
